In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/images2.zip /content/images2.zip

In [ ]:
!unzip /content/images2.zip -d /content

Streaming output truncated to the last 5000 lines.
  inflating: /content/images2/n596159818_US_20.jpg  
  inflating: /content/images2/n9927058853_US_21.jpg  
  inflating: /content/images2/a346180051_DE_.jpg  
  inflating: /content/images2/n1504909811_DE_.jpg  
  inflating: /content/images2/n1802392218_US_21.jpg  
  inflating: /content/images2/w353035122_US_18.jpg  
  inflating: /content/images2/w194922358_CH_21.jpg  
  inflating: /content/images2/n4786939050_DE_.jpg  
  inflating: /content/images2/n10028762885_US_20.jpg  
  inflating: /content/images2/a617030506_US_21.jpg  
  inflating: /content/images2/n8998885952_CH_20.jpg  
  inflating: /content/images2/n8919784777_DE_.jpg  
  inflating: /content/images2/n7745290275_CH_19.jpg  
  inflating: /content/images2/n5300700854_CH_19.jpg  
  inflating: /content/images2/w944518092_DE_.jpg  
  inflating: /content/images2/a92337905_CH_21.jpg  
  inflating: /content/images2/n8788925772_US_19.jpg  
  inflating: /content/images2/a380135411_S2_22.j

In [ ]:
!rm /content/images2.zip
!rm /content/drive/MyDrive/images2.zip

In [ ]:
!cp -r /content/images2 /content/drive/MyDrive/

In [ ]:
!pip install qwen-vl-utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 54.5 MB/s eta 0:00:00


In [ ]:
import torch
import os
import pandas as pd
from PIL import Image
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info

model_name = "Qwen/Qwen2-VL-2B-Instruct"
device = "cuda" if torch.cuda.is_available() else "cpu"

model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_name, torch_dtype="auto", device_map="auto"
).to(device)

processor = AutoProcessor.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/56.4k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/429M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [ ]:
image_folder = "/content" # "/content/drive/MyDrive"
csv_file_path = "/content/drive/MyDrive/SkyScript_train_unfiltered_5M.csv"
output_csv_file = "/content/drive/MyDrive/updated_annotations.csv"

In [ ]:
df_filtered = pd.read_csv(output_csv_file)

# df = pd.read_csv(csv_file_path)
# df_filtered = df[df["filepath"].str.contains("images2/")].reset_index(drop=True)

# # Eğer 'generated_captions' sütunu henüz eklenmemişse ekleyelim
# if "generated_captions" not in df_filtered.columns:
#     df_filtered["generated_captions"] = ""

# # Eğer output dosyası varsa, oradaki captionları input df'ye aktar
# if os.path.exists(output_csv_file):
#     print(f"{output_csv_file} dosyası mevcut, mevcut captionlar yükleniyor...")
#     df_out = pd.read_csv(output_csv_file)
#     if "generated_captions" in df_out.columns:
#         df_filtered["generated_captions"] = df_out["generated_captions"]


In [ ]:
prompt_text = "Generate a caption for the satellite image."

save_every = 100  

for idx, row in df_filtered.iterrows():
    # print("Idx: ", row)
    if pd.notna(row['generated_captions']) and str(row['generated_captions']).strip().lower() not in ["", "nan"]:
        print(f"ℹ️ {idx} numaralı satır atlanıyor: Caption mevcut.")
        continue

    img_file = row["filepath"]  
    img_path = os.path.join(image_folder, img_file)  

    if os.path.exists(img_path):
        image = Image.open(img_path).convert("RGB")

        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "image", "image": img_path},
                    {"type": "text", "text": prompt_text},
                ],
            }
        ]

        text_input = processor.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True
        )
        image_inputs, video_inputs = process_vision_info(messages)

        inputs = processor(
            text=[text_input],
            images=image_inputs,
            videos=video_inputs,
            padding=True,
            return_tensors="pt",
        ).to(device)

        with torch.no_grad():
            generated_ids = model.generate(**inputs, max_new_tokens=128)
            generated_ids_trimmed = [
                out_ids[len(in_ids):]
                for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
            ]
            caption = processor.batch_decode(
                generated_ids_trimmed,
                skip_special_tokens=True,
                clean_up_tokenization_spaces=False,
            )[0]

        df_filtered.at[idx, "generated_captions"] = caption
        print(f"✅ İşlenen {idx}: {img_file} → {caption}")

        if idx % save_every == 0 and idx > 0:
            df_filtered.to_csv(output_csv_file, index=False)
            print(f"💾 {output_csv_file} dosyası kaydedildi (Adım {idx}).")

df_filtered.to_csv(output_csv_file, index=False)
print("✅ Tüm captionlar üretildi ve güncellenmiş CSV dosyası kaydedildi.")

Streaming output truncated to the last 5000 lines.
✅ İşlenen 671031: images2/n4059139745_US_16.jpg → "An aerial view of a small residential area with houses and a road, surrounded by greenery."
✅ İşlenen 671032: images2/a33194962_L9_22.jpg → "An aerial view of a small town nestled amidst agricultural fields, with a central business district and scattered residential areas."
✅ İşlenen 671033: images2/w263534181_CH_20.jpg → "An aerial view of a parking lot with a few cars parked in a row, with a shadow cast by a nearby tree or structure."
✅ İşlenen 671034: images2/w16575728_CH_19.jpg → "An aerial view of a serene green landscape with a winding path cutting through the grass, surrounded by lush trees and a clear sky."
ℹ️ 671035 numaralı satır atlanıyor: Caption mevcut.
✅ İşlenen 671036: images2/n7289854749_CH_19.jpg → "An aerial view of a road intersection with a yellow line dividing the lanes and a small square in the center."
✅ İşlenen 671037: images2/a49178925_DE_.jpg → "An aerial view